In [59]:
# 熵权法确定会员完成能力
# 将../data/有效会员信息数据.xlsx中的每位会员的预订任务限额作为指标1
# 将每位会员的预订任务开始时间与6:30的差值分钟数（3的倍数）作为指标2
# 若指标2不为0，则指标2更新为自己的倒数
# 若指标2为0，则指标2统一更新为常数0.5
# x_kj表示第k个会员的指标j的值

import pandas as pd
import numpy as np
import math

In [60]:
# 读取数据
data = pd.read_excel('../data/有效会员信息数据.xlsx')
data = data.dropna()
data = data.reset_index(drop=True)

print(data)

       会员编号          经度         纬度  预订任务限额  预订任务开始时间
0     B0001  113.679983  22.947097     114  06:30:00
1     B0002  113.966524  22.577792     163  06:30:00
2     B0003  113.347272  23.192458     139  06:30:00
3     B0004  113.318750  23.255965      98  06:30:00
4     B0006  112.797680  22.262784      72  06:30:00
...     ...         ...        ...     ...       ...
1869  B1873  113.277245  22.840505       1  08:00:00
1870  B1874  113.287606  23.069415       1  08:00:00
1871  B1875  113.301736  23.333446       1  08:00:00
1872  B1876  113.994101  22.693506       1  08:00:00
1873  B1877  113.239864  23.133238       1  08:00:00

[1874 rows x 5 columns]


In [62]:
# 计算指标1
x_1 = data['预订任务限额'].values
print(x_1)

# 计算指标2
x_21 = data['预订任务开始时间'].values
x_2 = []
for i in range(len(x_21)):
    time_parts = x_21[i].split(':')
    minutes = int(time_parts[0]) * 60 + int(time_parts[1]) + int(time_parts[2]) / 60
    x_2.append(minutes)
x_2 = np.array(x_2) - 390


# 更新指标2
for i in range(len(x_2)):
    if x_2[i] != 0:
        x_2[i] = 1 / x_2[i]
    else:
        x_2[i] = 0.5
print(x_2)

# 用以上计算出的指标生成一份新的表格
data_new = pd.DataFrame()
data_new['会员编号'] = data['会员编号']
data_new['指标1'] = x_1
data_new['指标2'] = x_2
data_new.to_excel('../data/会员完成能力指标.xlsx', index=False)


[114 163 139 ...   1   1   1]
[0.5        0.5        0.5        ... 0.01111111 0.01111111 0.01111111]


In [63]:
# 采用Min-Max标准化方法对../data/会员完成能力指标.xlsx中的指标1和指标2分别进行归一化处理， 并把处理后的数据更新到原表格中
data = pd.read_excel('../data/会员完成能力指标.xlsx')
data = data.dropna()
data = data.reset_index(drop=True)

# 归一化处理
data['指标1'] = (data['指标1'] - data['指标1'].min()) / (data['指标1'].max() - data['指标1'].min())
data['指标2'] = (data['指标2'] - data['指标2'].min()) / (data['指标2'].max() - data['指标2'].min())
data.to_excel('../data/会员完成能力指标.xlsx', index=False)


In [64]:
# 分别计算每位会员指标1和指标2占所有会员的比重，并根据比重计算每个指标的熵值，从而得到信息熵冗余度，最终得到各指标的权值
data = pd.read_excel('../data/会员完成能力指标.xlsx')
data = data.dropna()
data = data.reset_index(drop=True)

# 计算指标1和指标2的比重
p_1 = data['指标1'].values / data['指标1'].sum()
p_2 = data['指标2'].values / data['指标2'].sum()

# 遍历会员编号进行比重计算操作
for i in range(len(data)):
    member_id = data.loc[i, '会员编号']
    data.loc[i, '指标1比重'] = p_1[i]
    data.loc[i, '指标2比重'] = p_2[i]

print(data)
# 存储比重数据
data.to_excel('../data/会员完成能力指标.xlsx', index=False)





       会员编号       指标1  指标2     指标1比重     指标2比重
0     B0001  0.489177  1.0  0.010392  0.008798
1     B0002  0.701299  1.0  0.014898  0.008798
2     B0003  0.597403  1.0  0.012691  0.008798
3     B0004  0.419913  1.0  0.008920  0.008798
4     B0006  0.307359  1.0  0.006529  0.008798
...     ...       ...  ...       ...       ...
1869  B1873  0.000000  0.0  0.000000  0.000000
1870  B1874  0.000000  0.0  0.000000  0.000000
1871  B1875  0.000000  0.0  0.000000  0.000000
1872  B1876  0.000000  0.0  0.000000  0.000000
1873  B1877  0.000000  0.0  0.000000  0.000000

[1874 rows x 5 columns]


In [65]:
import math

# 初始化熵值
e_1 = 0
e_2 = 0

p_1 = data['指标1比重'].values
p_2 = data['指标2比重'].values

# 常量K的计算
n = len(data)  # 样本数量
K = 1 / math.log(n)

# 计算每个指标的熵值
for i in range(n):
    if p_1[i] > 0:  # 避免 log(0) 错误
        e_1 += p_1[i] * math.log(p_1[i])
    if p_2[i] > 0:
        e_2 += p_2[i] * math.log(p_2[i])

# 熵值取负乘以K
e_1 = -e_1 * K
e_2 = -e_2 * K

# 计算信息熵冗余度
r_1 = 1 - e_1
r_2 = 1 - e_2

# 计算指标1和指标2的权值
w_1 = r_1 / (r_1 + r_2)
w_2 = r_2 / (r_1 + r_2)

# 输出权值
print('指标1的权值为:', w_1)
print('指标2的权值为:', w_2)

指标1的权值为: 0.369411047019983
指标2的权值为: 0.6305889529800169
